<div style="
    background: linear-gradient(135deg, #0c0c2e 0%, #1a1a4a 50%, #2d1a4a 100%);
    border: 2px solid #6366f1;
    border-radius: 15px;
    padding: 20px;
    margin: 15px 0;
    box-shadow: 0 0 25px rgba(99, 102, 241, 0.3),
                inset 0 0 15px rgba(255, 255, 255, 0.1);
    color: #e2e8f0;
    font-family: 'Segoe UI', system-ui, sans-serif;
    position: relative;
    overflow: hidden;
">

<div style="
    position: absolute;
    bottom: -30px;
    left: -30px;
    width: 80px;
    height: 80px;
    background: radial-gradient(circle, rgba(168, 85, 247, 0.3) 0%, transparent 70%);
    border-radius: 50%;
"></div>

<h1 style="
    color: #818cf8;
    margin-top: 0;
    text-align: center;
    font-weight: 600;
    text-shadow: 0 0 10px rgba(129, 140, 248, 0.5);
    position: relative;
    z-index: 1;
">
    Import libraries
</h1>

In [ ]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
import torch.nn as nn

from tqdm import tqdm
from scipy import signal
from scipy.signal import medfilt
from torch.utils.data import Dataset, DataLoader

<div style="
    background: linear-gradient(135deg, #0c0c2e 0%, #1a1a4a 50%, #2d1a4a 100%);
    border: 2px solid #6366f1;
    border-radius: 15px;
    padding: 20px;
    margin: 15px 0;
    box-shadow: 0 0 25px rgba(99, 102, 241, 0.3),
                inset 0 0 15px rgba(255, 255, 255, 0.1);
    color: #e2e8f0;
    font-family: 'Segoe UI', system-ui, sans-serif;
    position: relative;
    overflow: hidden;
">

<div style="
    position: absolute;
    bottom: -30px;
    left: -30px;
    width: 80px;
    height: 80px;
    background: radial-gradient(circle, rgba(168, 85, 247, 0.3) 0%, transparent 70%);
    border-radius: 50%;
"></div>

<h1 style="
    color: #818cf8;
    margin-top: 0;
    text-align: center;
    font-weight: 600;
    text-shadow: 0 0 10px rgba(129, 140, 248, 0.5);
    position: relative;
    z-index: 1;
">
    Config(u can change parameters)
</h1>

In [ ]:
class Config:
    batch_size = 16
    epochs = 1
    lr = 1e-3
    img_size = (512, 512)
    target_length = 5000

config = Config()

<div style="
    background: linear-gradient(135deg, #0c0c2e 0%, #1a1a4a 50%, #2d1a4a 100%);
    border: 2px solid #6366f1;
    border-radius: 15px;
    padding: 20px;
    margin: 15px 0;
    box-shadow: 0 0 25px rgba(99, 102, 241, 0.3),
                inset 0 0 15px rgba(255, 255, 255, 0.1);
    color: #e2e8f0;
    font-family: 'Segoe UI', system-ui, sans-serif;
    position: relative;
    overflow: hidden;
">

<div style="
    position: absolute;
    bottom: -30px;
    left: -30px;
    width: 80px;
    height: 80px;
    background: radial-gradient(circle, rgba(168, 85, 247, 0.3) 0%, transparent 70%);
    border-radius: 50%;
"></div>

<h1 style="
    color: #818cf8;
    margin-top: 0;
    text-align: center;
    font-weight: 600;
    text-shadow: 0 0 10px rgba(129, 140, 248, 0.5);
    position: relative;
    z-index: 1;
">
    ECG image processor: preprocessing + extraction of 12-lead signals
</h1>

In [ ]:
class ECGProcessor:
    
    def __init__(self):
        self.lead_regions = {
            'I': (0.1, 0.15), 'II': (0.1, 0.3), 'III': (0.1, 0.45),
            'aVR': (0.1, 0.6), 'aVL': (0.1, 0.75), 'aVF': (0.1, 0.9),
            'V1': (0.55, 0.15), 'V2': (0.55, 0.3), 'V3': (0.55, 0.45),
            'V4': (0.55, 0.6), 'V5': (0.55, 0.75), 'V6': (0.55, 0.9)
        }
    
    def adaptive_threshold(self, image):
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image
            
        mean_brightness = np.mean(gray)
        if mean_brightness > 200:
            return 170
        elif mean_brightness < 150:
            return 150
        else:
            return 160
    
    def preprocess_image(self, image):
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image
        
        threshold = self.adaptive_threshold(gray)
        _, binary = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
        
        kernel = np.ones((2, 2), np.uint8)
        cleaned = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
        
        return cleaned
    
    def extract_signal_trace(self, binary_image, lead_name):
        try:
            h, w = binary_image.shape
            x_ratio, y_ratio = self.lead_regions[lead_name]
            
            roi_width = int(w * 0.35)
            roi_height = int(h * 0.06)
            roi_x = int(w * x_ratio - roi_width // 2)
            roi_y = int(h * y_ratio - roi_height // 2)
            
            roi_x = max(0, roi_x)
            roi_y = max(0, roi_y)
            roi_width = min(roi_width, w - roi_x)
            roi_height = min(roi_height, h - roi_y)
            
            roi = binary_image[roi_y:roi_y+roi_height, roi_x:roi_x+roi_width]
            
            if roi.size == 0:
                return np.zeros(config.target_length)
            
            signal_positions = []
            for col in range(roi.shape[1]):
                column = roi[:, col]
                dark_indices = np.where(column < 128)[0]
                
                if len(dark_indices) > 0:
                    weights = 255 - column[dark_indices]
                    weighted_pos = np.average(dark_indices, weights=weights)
                    signal_positions.append(weighted_pos)
                else:
                    signal_positions.append(roi_height / 2)
            
            signal = np.array(signal_positions)
            
            signal = roi_height - signal
            signal = (signal - np.mean(signal)) / (np.std(signal) + 1e-8)
            
            if len(signal) > 1:
                signal = signal.resample(signal, config.target_length)
            
            signal = medfilt(signal, kernel_size=3)
            
            return signal.astype(np.float32)
            
        except Exception as e:
            print(f"Error extracting {lead_name}: {e}")
            return np.zeros(config.target_length)

<div style="
    background: linear-gradient(135deg, #0c0c2e 0%, #1a1a4a 50%, #2d1a4a 100%);
    border: 2px solid #6366f1;
    border-radius: 15px;
    padding: 20px;
    margin: 15px 0;
    box-shadow: 0 0 25px rgba(99, 102, 241, 0.3),
                inset 0 0 15px rgba(255, 255, 255, 0.1);
    color: #e2e8f0;
    font-family: 'Segoe UI', system-ui, sans-serif;
    position: relative;
    overflow: hidden;
">

<div style="
    position: absolute;
    bottom: -30px;
    left: -30px;
    width: 80px;
    height: 80px;
    background: radial-gradient(circle, rgba(168, 85, 247, 0.3) 0%, transparent 70%);
    border-radius: 50%;
"></div>

<h1 style="
    color: #818cf8;
    margin-top: 0;
    text-align: center;
    font-weight: 600;
    text-shadow: 0 0 10px rgba(129, 140, 248, 0.5);
    position: relative;
    z-index: 1;
">
    Electrocardiography Dataset
</h1>

In [ ]:
class ECGDataset(Dataset):
    
    def __init__(self, df, image_dir, is_train=True):
        self.df = df
        self.image_dir = image_dir
        self.is_train = is_train
        self.processor = ECGProcessor()
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        base_id = row['id']
        
        if self.is_train:
            img_folder = os.path.join(self.image_dir, str(base_id))
            img_files = [f for f in os.listdir(img_folder) if f.endswith('.png')]
            img_path = os.path.join(img_folder, img_files[0]) if img_files else None
        else:
            img_path = os.path.join(self.image_dir, f"{base_id}.png")
        
        image = cv2.imread(img_path) if img_path else None
        if image is None:
            image = np.ones((1240, 1024, 3), dtype=np.uint8) * 255
        
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_resized = cv2.resize(image_rgb, config.img_size)
        image_tensor = torch.from_numpy(image_resized.transpose(2, 0, 1)).float() / 255.0
        
        if self.is_train:
            try:
                csv_path = os.path.join(self.image_dir, str(base_id), f"{base_id}.csv")
                signals_df = pd.read_csv(csv_path)
                target_signal = signals_df['II'].values.astype(np.float32)
                
                if len(target_signal) != config.target_length:
                    target_signal = signal.resample(target_signal, config.target_length)
                
                target_signal = (target_signal - np.mean(target_signal)) / (np.std(target_signal) + 1e-8)
                
            except:
                t = np.linspace(0, 10, config.target_length)
                target_signal = (np.sin(2 * np.pi * 1 * t) + 
                               0.3 * np.sin(2 * np.pi * 2 * t))
                target_signal = target_signal.astype(np.float32)
            
            return image_tensor, torch.FloatTensor(target_signal), base_id
        else:
            return image_tensor, base_id

<div style="
    background: linear-gradient(135deg, #0c0c2e 0%, #1a1a4a 50%, #2d1a4a 100%);
    border: 2px solid #6366f1;
    border-radius: 15px;
    padding: 20px;
    margin: 15px 0;
    box-shadow: 0 0 25px rgba(99, 102, 241, 0.3),
                inset 0 0 15px rgba(255, 255, 255, 0.1);
    color: #e2e8f0;
    font-family: 'Segoe UI', system-ui, sans-serif;
    position: relative;
    overflow: hidden;
">

<div style="
    position: absolute;
    bottom: -30px;
    left: -30px;
    width: 80px;
    height: 80px;
    background: radial-gradient(circle, rgba(168, 85, 247, 0.3) 0%, transparent 70%);
    border-radius: 50%;
"></div>

<h1 style="
    color: #818cf8;
    margin-top: 0;
    text-align: center;
    font-weight: 600;
    text-shadow: 0 0 10px rgba(129, 140, 248, 0.5);
    position: relative;
    z-index: 1;
">
    Model architecture: Image to Signal Regression
</h1>

In [ ]:
class ECGNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((8, 8))
        )
        
        self.regressor = nn.Sequential(
            nn.Linear(128 * 8 * 8, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, config.target_length)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.regressor(x)
        return x

<div style="
    background: linear-gradient(135deg, #0c0c2e 0%, #1a1a4a 50%, #2d1a4a 100%);
    border: 2px solid #6366f1;
    border-radius: 15px;
    padding: 20px;
    margin: 15px 0;
    box-shadow: 0 0 25px rgba(99, 102, 241, 0.3),
                inset 0 0 15px rgba(255, 255, 255, 0.1);
    color: #e2e8f0;
    font-family: 'Segoe UI', system-ui, sans-serif;
    position: relative;
    overflow: hidden;
">

<div style="
    position: absolute;
    bottom: -30px;
    left: -30px;
    width: 80px;
    height: 80px;
    background: radial-gradient(circle, rgba(168, 85, 247, 0.3) 0%, transparent 70%);
    border-radius: 50%;
"></div>

<h1 style="
    color: #818cf8;
    margin-top: 0;
    text-align: center;
    font-weight: 600;
    text-shadow: 0 0 10px rgba(129, 140, 248, 0.5);
    position: relative;
    z-index: 1;
">
    ECG Loss Function
</h1>

In [ ]:
class ECGLoss(nn.Module):
    
    def __init__(self):
        super().__init__()
        
    def forward(self, pred, target):
        mse_loss = nn.MSELoss()(pred, target)
        
        pred_diff = pred[:, 1:] - pred[:, :-1]
        target_diff = target[:, 1:] - target[:, :-1]
        smooth_loss = nn.MSELoss()(pred_diff, target_diff)
        
        return mse_loss + 0.1 * smooth_loss

<div style="
    background: linear-gradient(135deg, #0c0c2e 0%, #1a1a4a 50%, #2d1a4a 100%);
    border: 2px solid #6366f1;
    border-radius: 15px;
    padding: 20px;
    margin: 15px 0;
    box-shadow: 0 0 25px rgba(99, 102, 241, 0.3),
                inset 0 0 15px rgba(255, 255, 255, 0.1);
    color: #e2e8f0;
    font-family: 'Segoe UI', system-ui, sans-serif;
    position: relative;
    overflow: hidden;
">

<div style="
    position: absolute;
    bottom: -30px;
    left: -30px;
    width: 80px;
    height: 80px;
    background: radial-gradient(circle, rgba(168, 85, 247, 0.3) 0%, transparent 70%);
    border-radius: 50%;
"></div>

<h1 style="
    color: #818cf8;
    margin-top: 0;
    text-align: center;
    font-weight: 600;
    text-shadow: 0 0 10px rgba(129, 140, 248, 0.5);
    position: relative;
    z-index: 1;
">
    Model training
</h1>

In [ ]:
train_df = pd.read_csv('/kaggle/input/physionet-ecg-image-digitization/train.csv')
train_df = train_df.head(800)

train_dataset = ECGDataset(
    train_df, 
    '/kaggle/input/physionet-ecg-image-digitization/train',
    is_train=True
)

train_loader = DataLoader(
    train_dataset, 
    batch_size=config.batch_size, 
    shuffle=True, 
    num_workers=2
)
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ECGNet().to(device)

if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
    
criterion = ECGLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config.epochs)
    
best_loss = float('inf')
for epoch in range(config.epochs):
    model.train()
    running_loss = 0.0
    
    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{config.epochs}')
    for images, targets, _ in pbar:
        images, targets = images.to(device), targets.to(device)
            
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
            
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
            
        running_loss += loss.item()
        pbar.set_postfix({'Loss': f'{loss.item():.4f}'})
        
    epoch_loss = running_loss / len(train_loader)
    scheduler.step()
    
    print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.6f}')
    
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        torch.save(model.state_dict(), 'ecg_model.pth')

<div style="
    background: linear-gradient(135deg, #0c0c2e 0%, #1a1a4a 50%, #2d1a4a 100%);
    border: 2px solid #6366f1;
    border-radius: 15px;
    padding: 20px;
    margin: 15px 0;
    box-shadow: 0 0 25px rgba(99, 102, 241, 0.3),
                inset 0 0 15px rgba(255, 255, 255, 0.1);
    color: #e2e8f0;
    font-family: 'Segoe UI', system-ui, sans-serif;
    position: relative;
    overflow: hidden;
">

<div style="
    position: absolute;
    bottom: -30px;
    left: -30px;
    width: 80px;
    height: 80px;
    background: radial-gradient(circle, rgba(168, 85, 247, 0.3) 0%, transparent 70%);
    border-radius: 50%;
"></div>

<h1 style="
    color: #818cf8;
    margin-top: 0;
    text-align: center;
    font-weight: 600;
    text-shadow: 0 0 10px rgba(129, 140, 248, 0.5);
    position: relative;
    z-index: 1;
">
    Predict, create submission and check results
</h1>

In [ ]:
test_df = pd.read_csv('/kaggle/input/physionet-ecg-image-digitization/test.csv')
test_dataset = ECGDataset(test_df, '/kaggle/input/physionet-ecg-image-digitization/test', is_train=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

device = next(model.parameters()).device
model.eval()

submission_data = []

for batch_idx, (images, base_ids) in enumerate(tqdm(test_loader, desc='Predicting')):
    if batch_idx >= len(test_df):
        break
        
    test_row = test_df.iloc[batch_idx]
    base_id = test_row['id']
    lead = test_row['lead']
    num_rows = test_row['number_of_rows']
    
    images = images.to(device)
    
    with torch.no_grad():
        prediction = model(images).cpu().numpy().flatten()
    
    if len(prediction) > num_rows:
        prediction = prediction[:num_rows]
    else:
        prediction = np.pad(prediction, (0, num_rows - len(prediction)), mode='edge')
        
    prediction = medfilt(prediction, kernel_size=3)
        
    for row_id in range(num_rows):
        composite_id = f"{base_id}_{row_id}_{lead}"
        submission_data.append({
            'id': composite_id,
            'value': float(prediction[row_id])
        })

submission = pd.DataFrame(submission_data)
submission.to_csv('submission.csv', index=False)
submission.head(30)